In [1]:
import pandas as pd
import numpy as np
import gc 
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)
from sklearn.preprocessing import Imputer

In [2]:
def load_data(path_to_data='data', sample_size = None):
    logging.info('Reading Properties 2016...')
    prop_2016 = pd.read_csv('{}/properties_2016.csv'.format(path_to_data))
    
    logging.info('Reading Properties 2017...')
    prop_2017 = pd.read_csv('{}/properties_2017.csv'.format(path_to_data))
    
    logging.info('Reading Train 2016...')
    target_2016 = pd.read_csv('{}/train_2016_v2.csv'.format(path_to_data))
    
    logging.info('Reading Train 2017..')
    target_2017 = pd.read_csv('{}/train_2017.csv'.format(path_to_data))
    
    logging.info('Performing merge')
    joined_data_2016 = pd.merge(target_2016,prop_2016,on="parcelid",how="left")
    joined_data_2017 = pd.merge(target_2017, prop_2017,on='parcelid',how='left')

    joined_data = pd.concat([joined_data_2016,joined_data_2017])

    # convert dates:
    joined_data.transactiondate = pd.to_datetime(joined_data.transactiondate,format="%Y-%m-%d")

    joined_data['transaction_mth'] = joined_data.transactiondate.apply(lambda x:x.month)
    joined_data['transaction_yr'] = joined_data.transactiondate.apply(lambda x: x.year)
    joined_data['transaction_day_of_wk'] = joined_data.transactiondate.apply(lambda x: x.dayofweek)
    joined_data=joined_data.drop('transactiondate',axis=1)
    
    del target_2016
    del target_2017
    del prop_2016
    del prop_2017

    gc.collect()
    
#    if sample_size is not None:
#        logging.info('Sampling: {} of data'.format(sample_size))
#        joined_data = joined_data.sample(frac=sample_size)

    return joined_data, joined_data['logerror'].values

In [ ]:
zip_data = pd.read_csv('C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/CA_zip_city.csv')

In [ ]:
list(joined_data.columns)

In [ ]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)

    return props, NAlist

In [ ]:
def calculate_missing(joined_data):
    ## calculatedbathnbr same as bathroomcnt and has more nas
    joined_data[joined_data.bathroomcnt !=joined_data.calculatedbathnbr][['bathroomcnt', 'calculatedbathnbr']].dropna()
    joined_data[['bathroomcnt', 'calculatedbathnbr']].isnull().sum()

    #Fill in those properties that have a pool with median pool value
    poolsizesum_median = joined_data.loc[joined_data['poolsizesum'] > 0, 'poolsizesum'].median()
    joined_data.loc[(joined_data['poolsizesum'].isnull() | 
                    joined_data['poolsizesum'] == 0) & 
                ((pd.notnull(joined_data['poolcnt'])) |
                (pd.notnull(joined_data['pooltypeid10'])) |
                (pd.notnull(joined_data['pooltypeid2'])) |
                (pd.notnull(joined_data['pooltypeid7']))), 'poolsizesum'] = poolsizesum_median

    #Fill in those properties that have a pool size/type with pool count of 1
    joined_data.loc[(joined_data['poolcnt'] != 1) & 
                ((pd.notnull(joined_data['poolsizesum'])) |
                (pd.notnull(joined_data['pooltypeid10'])) |
                (pd.notnull(joined_data['pooltypeid2'])) |
                (pd.notnull(joined_data['pooltypeid7']))), 'poolcnt'] = 1

    # fill in fireplace count with mode when there is a fireplace flag is true
    joined_data.loc[(joined_data.fireplaceflag == True) & 
                    (joined_data.fireplacecnt.isnull()), 'fireplacecnt'] = joined_data.fireplacecnt.dropna().mode()[0]

    # number of car garage is more that sqft
    garage_median = joined_data[(joined_data.garagetotalsqft != 0)].garagetotalsqft.dropna().median()
    joined_data.loc[(joined_data.garagetotalsqft == 0) & (joined_data.garagecarcnt > 0), 'garagetotalsqft'] = garage_median

    # unitcnt extreme outliers
    unit_median = joined_data.loc[pd.notnull(joined_data['unitcnt'])]['unitcnt'].median()
    joined_data.loc[(joined_data['unitcnt'] > 9), 'unitcnt'] = unit_median

    return joined_data

In [ ]:
def drop_columns(data, drop_cols):
    # mostly null
    data = data.drop(drop_cols, axis=1)
    data.drop_duplicates(inplace = True)

    return data

In [ ]:
def columns_after_drop(numeric, categorical, drop_columns):
    numeric = list(set(numeric) - (set(numeric) & set(drop_columns)))
    categorical = list(set(categorical) - (set(categorical) & set(drop_columns)))

    return numeric, categorical

In [ ]:
def impute_numerical_var(joined_data, numerical_cols, imputation= None):
    logging.info('Filling numeric NAs')
    if imputation:
        for col, val in imputations_numeric.items():
            if col in properties.columns:
                properties[col].fillna(val, inplace=True)
                return properties
    else:
    # numerical vars
        numerical_data = joined_data.copy().reset_index()
        numerical_data = numerical_data[numerical_cols]
        numerical_data_cols = numerical_data.columns

        numeric_imp  = Imputer(strategy='median', axis=0)     
        numerical_data = pd.DataFrame(numeric_imp.fit_transform(numerical_data.values), columns=numerical_data_cols)

        return numerical_data,  {key:val for key,val in  zip(numerical_data_cols, numeric_imp.statistics_)}

In [ ]:
def impute_categorical_var(joined_data, categorical_cols):
    # categorical vars
    categorical_data = joined_data.copy().reset_index()
    categorical_data  = categorical_data[categorical_cols]

    if 'hashottuborspa' in categorical_cols:
        categorical_data['hashottuborspa']=categorical_data['hashottuborspa'].apply(lambda x: 1 if x == 'True' else 0)

    if 'taxdelinquencyflag' in categorical_cols:
        categorical_data['taxdelinquencyflag']=categorical_data['taxdelinquencyflag'].apply(lambda x: 1 if str(x).strip().lower() == 'y' else 0)

    for c, dtype in zip(categorical_data.columns, categorical_data.dtypes):
        categorical_data[c] = categorical_data[c].apply(lambda x: x if pd.isnull(x) else str(x))

    categorical_data_cols = categorical_data.columns

    most_frequent_lst = []
    
    logging.info('Using most frequent...')
    
    for col in categorical_data_cols:
        logging.info("Filling NA: {}".format(col))
        # logging.info("Filling NA: {}".format(col))
        mk=categorical_data[col].notnull()
        value_counts = categorical_data[mk][col].value_counts()
        most_frequent_lst.append(value_counts.index[0])
        categorical_data[col].fillna(most_frequent_lst[-1], inplace=True)

    return categorical_data, {key:val for key,val in zip(categorical_data_cols, most_frequent_lst)}

In [ ]:
numeric_cols = ['parcelid', 'assessmentyear','basementsqft',	'bathroomcnt',	'bedroomcnt',	'calculatedbathnbr', 
                'calculatedfinishedsquarefeet',	'finishedfloor1squarefeet',	'finishedsquarefeet12',
                'finishedsquarefeet13',	'finishedsquarefeet15',	'finishedsquarefeet50',	'finishedsquarefeet6',
                'fireplacecnt',	'fullbathcnt',	'garagecarcnt',	'garagetotalsqft',	'landtaxvaluedollarcnt',
                'lotsizesquarefeet',	'numberofstories',	'poolcnt',	'poolsizesum',	'roomcnt',
                'structuretaxvaluedollarcnt',	'taxamount',	'taxvaluedollarcnt',	'threequarterbathnbr',
                'unitcnt',	'yardbuildingsqft17',	'yardbuildingsqft26','transaction_day_of_wk','transaction_mth','transaction_yr',
               'taxdelinquencyyear','yearbuilt','latitude','longitude']

categorical_cols = ['parcelid', 'airconditioningtypeid','architecturalstyletypeid','buildingclasstypeid','buildingqualitytypeid',
                'censustractandblock','decktypeid','fips','fireplaceflag','hashottuborspa',
                 'heatingorsystemtypeid','pooltypeid10','pooltypeid2','pooltypeid7','propertycountylandusecode',
                 'propertylandusetypeid','propertyzoningdesc', 'rawcensustractandblock','regionidcity','regionidcounty','regionidneighborhood','regionidzip',
                 'storytypeid','taxdelinquencyflag','typeconstructiontypeid']

drop_cols = ['buildingclasstypeid','propertyzoningdesc',	'numberofstories',	'threequarterbathnbr',	
            'finishedfloor1squarefeet','finishedsquarefeet50','finishedsquarefeet15',
            'finishedsquarefeet12', 'yardbuildingsqft17',	'finishedsquarefeet6',	'yardbuildingsqft26',	
            'basementsqft',	'finishedsquarefeet13','assessmentyear','calculatedbathnbr',
            'rawcensustractandblock', 'regionidzip','regionidcounty','regionidcity','regionidneighborhood',
            'regionidneighborhood','taxvaluedollarcnt','buildingclasstypeid','storytypeid']	


# dropped columns removed: 
# 'garagetotalsqft',	'garagecarcnt',	'poolcnt','fireplacecnt',	'poolsizesum', 'censustractandblock',
# 'fireplaceflag',

In [ ]:
# use pipeline
joined_data, logerror_var = load_data(path_to_data='data')

In [ ]:
pd.DataFrame.to_csv(joined_data, 'C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/joined_data.csv')

In [ ]:
# use pipeline
joined_data, logerror_var = load_data(path_to_data='data')
# joined_data = drop_columns(joined_data, drop_cols)
joined_data, NAlist = reduce_mem_usage(joined_data)
joined_data = calculate_missing(joined_data)

In [ ]:
# if you need to drop a column, add it to drop_cols
# numeric_cols, categorical_cols = columns_after_drop(numeric_cols, categorical_cols, drop_cols)
numeric_data, imputations_numeric = impute_numerical_var(joined_data, numeric_cols)
categorical_data, imputations_categorical = impute_categorical_var(joined_data, categorical_cols)

In [ ]:
pd.merge(numeric_data, categorical_data, on="parcelid", how="left")

In [ ]:
num_cat_data = pd.merge(numeric_data, categorical_data, on="parcelid", how="left")

In [ ]:
joined_data = pd.merge(num_cat_data, joined_data[['parcelid', 'logerror']],on='parcelid',how='left')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import time

In [ ]:
num_cols = list(set(numeric_cols) - set(drop_cols))
train, test = train_test_split(joined_data, test_size=0.2,random_state=4)

train = train.sample(frac=0.01)
X_train = train[num_cols].drop('parcelid',axis=1)
y_train = train['logerror']

In [ ]:
rf_feat = RandomForestRegressor(n_jobs=-1,criterion='mae', random_state=4)

In [ ]:
len(train)

In [ ]:
start = time.time()
rf_feat.fit(X_train,y_train)
stop = time.time()
print("Elapsed_time: {}".format(stop - start))

In [ ]:
# When we use 1,343 observations
for numeric_feature, score in sorted(zip(rf_feat.feature_importances_,X_train.columns),reverse=True):
    print(numeric_feature, score)

In [ ]:
categorical_cols = categorical_cols[1:]
cat_train = train[categorical_cols].copy()
cat_dummies = pd.get_dummies(cat_train[categorical_cols])

In [ ]:
cat_train[categorical_cols].columns

In [ ]:
rf_feat.fit(cat_dummies, y_train)

In [ ]:
# When we use 13,431 observations
for numeric_feature, score in sorted(zip(rf_feat.feature_importances_,cat_dummies.columns),reverse=True):
    print(numeric_feature, score)